In [1]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
import natsort
from openpyxl import load_workbook
import scipy.io

In [8]:
#admin stuff
# filenames must be of type label_h_*_filenam, where * is a certain impulse response legnth
# e.g. '180_h_10_music-hd-0006'
newFileName = 'D:/Users/MC/Documents/UNI/MASTER/thesis/MFCC_FEATURES2/Mel_scale/data7_unseen_100ms.csv' #don't forget the '.csv'
# newFileLocation = '' #empyt if current folder suffices
readAudioFrom = 'D:/Users/MC/Documents/UNI/MASTER/thesis/AUGMENTED_DATA/data7_unseen'
impulseResponseLength = 'h_100'

In [9]:
#preparing header
# consist of original filename the mean and variance of each Mel frequency bank, the mean and variance of each corrseponding delta and delta-delta
header = 'filename '
for i in range(0, 40):
    header += f' mfccM{i}'
    header += f' mfccV{i}'
for i in range(0, 40):
    header += f' deltaM{i}'
    header += f' deltaV{i}'
for i in range(0, 40):
    header += f' deltaDeltaM{i}'
    header += f' deltaDeltaV{i}'
header += ' label'
header = header.split()
file = open(newFileName, 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [10]:
#reading dataset from csv
data = pd.read_csv(newFileName)
data.head()

,filename,mfccM0,mfccV0,mfccM1,mfccV1,mfccM2,mfccV2,mfccM3,mfccV3,mfccM4,...,deltaDeltaV35,deltaDeltaM36,deltaDeltaV36,deltaDeltaM37,deltaDeltaV37,deltaDeltaM38,deltaDeltaV38,deltaDeltaM39,deltaDeltaV39,label


In [11]:
#READING .wav
directions = '0 9 18 27 36 45 54 63 72 81 90 99 108 117 126 135 144 153 162 171 180 189 198 207 216 225 234 243 252 261 270 279 288 297 306 315 324 333 342 351'.split()
for d in directions:
    print(d)
    for filename in os.listdir(f'{readAudioFrom}/{d}'):
        if filename.startswith(f"{d}_{impulseResponseLength}_"):
            #get audio data
            audio_file = f'{readAudioFrom}/{d}/{filename}'
            y,sr = librosa.load(audio_file, sr=44100)
            
            #get features
            fftWindowSize = 1024                 #~23ms (default:2048 =~46ms @44.1kHz)
            overlapSize = int(fftWindowSize/4)   #ovelap 75%
            numOfMelFilterBanks = 40             #amount of Mel filterbanks
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=numOfMelFilterBanks, n_mels = numOfMelFilterBanks, n_fft=fftWindowSize, hop_length=overlapSize)
#             mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=numOfMelFilterBanks, n_fft=fftWindowSize, hop_length=overlapSize)
            delta = librosa.feature.delta(mfcc)
            deltaDelta = librosa.feature.delta(mfcc, order=2)
            
            #write to csv
            to_append = f'{filename} ' 
            #average psd per filterbank and write to file
            for i in mfcc:
                to_append += f' {np.mean(i)}' 
                to_append += f' {np.var(i)}' 
            #average speed
            for j in delta:
                to_append += f' {np.mean(j)}'
                to_append += f' {np.var(j)}'
            #average acceleration
            for k in deltaDelta:
                to_append += f' {np.mean(k)}'
                to_append += f' {np.var(k)}'
            #write label to file
            to_append += f' {d}'
            file = open(newFileName, 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

0
9
18
27
36
45
54
63
72
81
90
99
108
117
126
135
144
153
162
171
180
189
198
207
216
225
234
243
252
261
270
279
288
297
306
315
324
333
342
351


In [12]:
#reading dataset from csv
data = pd.read_csv(newFileName)
data.head()

,filename,mfccM0,mfccV0,mfccM1,mfccV1,mfccM2,mfccV2,mfccM3,mfccV3,mfccM4,...,deltaDeltaV35,deltaDeltaM36,deltaDeltaV36,deltaDeltaM37,deltaDeltaV37,deltaDeltaM38,deltaDeltaV38,deltaDeltaM39,deltaDeltaV39,label
0,0_h_100_speech-librivox-0045.wav,-208.762604,3607.959473,113.870377,1612.701904,-4.249023,523.207825,25.761396,276.531860,18.961596,...,0.034175,0.000652,0.026909,0.002094,0.035421,0.000565,0.032910,0.001051,0.026664,0
1,0_h_100_speech-librivox-0048.wav,-292.260864,9578.547852,46.388550,3547.663818,0.103560,211.293396,14.265685,369.503632,-0.238423,...,0.012128,-0.000166,0.012053,0.000072,0.015334,-0.001050,0.015348,0.000338,0.015308,0
2,0_h_100_speech-librivox-0063.wav,-212.106659,4401.036133,84.419121,2086.995605,-11.552585,670.546692,17.846823,510.897705,11.416938,...,0.043111,0.000716,0.038404,0.000538,0.033578,0.000029,0.031634,0.000998,0.035085,0
3,0_h_100_speech-librivox-0064.wav,-230.758011,6515.092773,74.299461,2490.360107,-5.919938,540.635376,22.162445,283.911560,14.708876,...,0.028868,0.000263,0.025760,-0.000036,0.026685,-0.000092,0.023596,0.003217,0.023691,0
4,0_h_100_speech-librivox-0089.wav,-190.672516,2728.310791,116.256783,805.050781,-11.611468,448.073730,16.122904,193.566879,14.517994,...,0.028909,-0.001850,0.028423,-0.000850,0.028519,0.000060,0.028405,-0.000097,0.027107,0


In [12]:
#check for discrepancy between filename and file contents
# a possible error could be that the file with name 'data5_1000ms.csv' contains an entry '0_h_50_music-fma-0006.wav'
for filename in os.listdir('D:/Users/MC/Documents/UNI/MASTER/thesis/MFCC_FEATURES2/Mel_scale'):
    print(filename)
    split_filename_list = filename.split("_")
    filename_list_end = split_filename_list[1].split("ms")
    filter = f'h_{filename_list_end[0]}_'
    dataFileEntry = pd.read_csv(os.path.join('D:/Users/MC/Documents/UNI/MASTER/thesis/MFCC_FEATURES2/Mel_scale',filename))
    entryName = dataFileEntry['filename'].iloc[0]
    if filter not in entryName:
        print('\nERROR')
        print(f'filename is "{filename}" yet contains {entryName}')
print('all files checked')

data5_1000ms.csv
data5_100ms.csv
data5_10ms.csv
data5_500ms.csv
data5_50ms.csv
data5_unseen_1000ms.csv

ERROR
filename is "data5_unseen_1000ms.csv" yet contains 0_h_1000_1195-130164-0023.wav
data5_unseen_100ms.csv

ERROR
filename is "data5_unseen_100ms.csv" yet contains 0_h_100_1195-130164-0023.wav
data5_unseen_10ms.csv

ERROR
filename is "data5_unseen_10ms.csv" yet contains 0_h_10_1195-130164-0023.wav
data5_unseen_500ms.csv

ERROR
filename is "data5_unseen_500ms.csv" yet contains 0_h_500_1195-130164-0023.wav
data5_unseen_50ms.csv

ERROR
filename is "data5_unseen_50ms.csv" yet contains 0_h_50_1195-130164-0023.wav
data5_validation_1000ms.csv

ERROR
filename is "data5_validation_1000ms.csv" yet contains 0_h_1000_101-126860-0049.wav
data5_validation_100ms.csv

ERROR
filename is "data5_validation_100ms.csv" yet contains 0_h_100_101-126860-0049.wav
data5_validation_10ms.csv

ERROR
filename is "data5_validation_10ms.csv" yet contains 0_h_10_101-126860-0049.wav
data5_validation_500ms.csv

ERR

In [13]:
len(data)/40

32.0